In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.integrate import odeint
import math


In [2]:
font = {'size'   : 10, 'family':'STIXGeneral'}
axislabelfontsize='small'
matplotlib.rc('font', **font)
matplotlib.mathtext.rcParams['legend.fontsize']='medium'

In [3]:
LCDM_settings = {# wich output? ClTT, transfer functions delta_i and theta_i
                   'output':'tCl,pCl,lCl,mPk',
                   'lensing':'yes',
                   # LambdaCDM parameters
                   'h':0.6821,
                   'omega_b':0.02253,
                   'omega_cdm':0.1177,
                   'A_s':2.216e-9, #caught a typo
                   'n_s':0.9686,
                   'tau_reio':0.085,
                   'm_ncdm': 0.06,
                   'N_ncdm':1, 
                   'N_ur':2.0328,
                   #'Omega_Lambda': 6.878622e-01,
                   # Take fixed value for primordial Helium (instead of automatic BBN adjustment)
                   #'YHe':0.246,
                   # other output and precision parameters
                   'l_max_scalars':2508,
                   'non linear':'HMcode',
                   'P_k_max_h/Mpc':100,
                   'z_max_pk':2.}

In [4]:
LCDM = Class()
LCDM.set(LCDM_settings)
LCDM.compute()

In [28]:
del EDE

In [70]:
EDE = Class()
#  Use best-fit params  of Smith+ :
EDE.set({'h': .7219})
EDE.set({'f_scf': 3.975e+26, 'm_scf': 5.4e-28,'thetai_scf': 2.83, 'A_s': 2.215e-09, 'n_s': 0.9889, 'omega_b': 0.02253, 'omega_cdm': 0.1306, 'm_ncdm': 0.06, 'tau_reio': 0.072})
EDE.set({'non linear':'HMCODE','N_ncdm':1, 'N_ur':2.0328, 'Omega_Lambda':0.0, 'Omega_fld':0, 'Omega_scf':-1, 'n_scf':3, 'CC_scf':1, 'scf_parameters':'1, 1, 1, 1, 1, 0.0', 'scf_tuning_index':3, 'attractor_ic_scf':'no', 'output':'tCl pCl lCl mPk', 'lensing':'yes', 'l_max_scalars':2508, 'P_k_max_h/Mpc':30,'z_max_pk':2.})
EDE.compute()

In [71]:
#'f_scf': 3.985e+26, 'm_scf': 5.4e-28. Extremeley close!!
print np.log10(EDE.z_c())
print EDE.theta_s_100()
print EDE.fEDE()

3.565765487761381
1.04151119761
0.122210630374


In [59]:
# m=5.5e-28, 'f_scf': 3.985e+26
print np.log10(EDE.z_c())
print EDE.theta_s_100()
print EDE.fEDE()

3.590027257769763
1.04158192306
0.123543244082


In [43]:
# m=5.295e-28
print np.log10(EDE.z_c())
print EDE.theta_s_100()
print EDE.fEDE()

3.5596994867891287
1.04141985453
0.122344780867


In [45]:
10**3.5596994867891287 - 10**3.562732440197178

-25.42718445345372

In [35]:
# m=5.3
print np.log10(EDE.z_c())
print EDE.theta_s_100()
print EDE.fEDE()

3.562732440197178
1.04142144065
0.12235075021


In [6]:
# Cell
# note Cl^gamma-gamma = 1/4 * [(l+2)!/(l-2)!] C_l^phi-phi.

clLCDM=LCDM.lensed_cl()
clEDE=EDE.lensed_cl()
clell = clLCDM.get("ell")
clTTLCDM = clLCDM.get("tt")
clTTEDE = clEDE.get("tt")
clppLCDM = clLCDM.get("pp")
clppEDE = clEDE.get("pp")
clEELCDM = clLCDM.get("ee")
clEEEDE = clEDE.get("ee")
clTELCDM = clLCDM.get("te")
clTEEDE = clEDE.get("te")




DellTTLCDM = 1e12*np.multiply(np.multiply(clell,clell+1),clTTLCDM)/(2*3.14)
DellTTEDE = 1e12*np.multiply(np.multiply(clell,clell+1),clTTEDE)/(2*3.14)
DellppLCDM = 1e12*np.multiply(np.multiply(clell,clell+1),clppLCDM)/(2*3.14)
DellppEDE = 1e12*np.multiply(np.multiply(clell,clell+1),clppEDE)/(2*3.14)
DellTELCDM = 1e12*np.multiply(np.multiply(clell,clell+1),clTELCDM)/(2*3.14)
DellTEEDE = 1e12*np.multiply(np.multiply(clell,clell+1),clTEEDE)/(2*3.14)
DellEELCDM = 1e12*np.multiply(np.multiply(clell,clell+1),clEELCDM)/(2*3.14)
DellEEEDE = 1e12*np.multiply(np.multiply(clell,clell+1),clEEEDE)/(2*3.14)









In [14]:
print DellTTLCDM

[  0.           0.         146.84808581 ...  10.35720517  10.32769356
  10.29838345]


In [9]:
# note Cl^gamma-gamma = 1/4 * [(l+2)!/(l-2)!] C_l^phi-phi.
# Note: [(l+2)!/(l-2)!]= (l+2)(l+1)(l)(l-1)
cellgg = np.zeros(len(clell)-1)
DellggLCDM = np.zeros(len(clell)-1)
DellggEDE= np.zeros(len(clell)-1)
DiffTT= np.zeros(len(clell)-1)

for i in range(2,len(clell)-1):
    cellgg[i]=clell[i]
    prefactor=0.25*1e12*(clell[i]+2)*(clell[i]+1)*clell[i]*(clell[i]-1)
    DellggLCDM[i] = 0.25*1e12*(math.factorial(clell[i]+2)/math.factorial(clell[i]-2))*clell[i]*(clell[i]+1)*clppLCDM[i]/(2*3.14)
    DellggEDE[i] = 0.25*1e12*(math.factorial(clell[i]+2)/math.factorial(clell[i]-2))*clell[i]*(clell[i]+1)*clppEDE[i]/(2*3.14)
    DiffTT[i] = clTTEDE[i]/clTTLCDM[i] - 1
 

In [10]:
# TT, EE, TE, lensing
plt.figure()
plt.plot(clell,DellTTLCDM,c='k',lw=1.)
plt.plot(clell,DellTTEDE,c='r',lw=1.)
plt.xscale('linear')
plt.yscale('log')
plt.xlim([2,2500])
plt.ylim([1e1,1e3])
plt.xlabel(r'$\ell$')
plt.ylabel(r'$D_\ell ^{\rm TT} [\mu {\rm K}^2 ]$')
plt.savefig('DellTT.pdf')
plt.clf()

plt.figure()
plt.plot(clell,DellTELCDM,c='k')
plt.plot(clell,DellTEEDE,c='r')
plt.xscale('linear')
plt.yscale('linear')
plt.xlim([2,2500])
plt.ylabel(r'$D_\ell ^{\rm TE} [\mu {\rm K}^2 ]$')
plt.savefig('DellTE.pdf')
plt.clf()

plt.figure()
plt.plot(clell,DellEELCDM,c='k')
plt.plot(clell,DellEEEDE,c='r')
plt.xlim([2,2500])
plt.xscale('linear')
plt.yscale('log')
plt.ylabel(r'$D_\ell ^{\rm EE} [\mu {\rm K}^2 ]$')
plt.savefig('DellEE.pdf')
plt.clf()

plt.figure()
plt.plot(cellgg,DellggLCDM,c='k')
plt.plot(cellgg,DellggEDE,c='r')
plt.xlim([2,2500])
plt.xscale('linear')
#plt.yscale('log')
plt.ylabel(r'$D_\ell ^{\gamma \gamma} [\mu {\rm K}^2 ]$')
plt.savefig('Dellgg.pdf')
plt.clf()




In [11]:
#Difference Plot
plt.figure()
plt.plot(cellgg,DiffTT,c='k',lw=1.)
plt.xscale('log')
plt.yscale('linear')
plt.xlim([2,2500])
plt.ylim([-.1,.1])
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\Delta C_\ell ^{\rm TT}/C_\ell [\mu {\rm K}^2 ]$')
plt.grid()
plt.savefig('DiffTT.pdf')
plt.clf()

In [12]:
print LCDM.Omega0_m()
print EDE.Omega0_cdm()
print EDE.Omega_b()
print EDE.omega_b()
print EDE.Omega_m()

0.302785564546
0.250604631646
0.0432321772663
0.02253
0.295072259621


In [55]:
EDE.Omega_m()*EDE.h()**2

1.6230903043637963